In [1]:
import os
from crewai import Agent, Task, Crew


# Define the Agents
analista_de_edital = Agent(
    role="Analista de Edital",
    goal=(
        "Extrair e categorizar os trechos mais relevantes do edital SOMENTE para a categoria Aliança Educacional, incluindo, mas não se restringindo prazos, critérios financeiros e condições de participação." 
    ),
    backstory=(
        "Um especialista em editais brasileiros de subvenção econômica para inovação. Detalhista e metódico, extrai com precisão os trechos mais relevantes do edital para que a Cogmo, uma startup brasileira de IA, possa utilizar essas informações para submeter uma proposta."
    ),
    model="gpt-4o",
    temperature=0.4,
    verbose=True,
    allow_delegation=False,
)

analista_de_requisitos = Agent(
    role="Levantador de Requisitos",
    goal=(
        "Identificar em cada trecho do material preparado pelo Analista de Edital se algum requisito de participação ou submissão de proposta foi mencionado."
        "Se houver algum requisito, este deve ser categorizado descrito como um item de uma lista de verificação."
    ),
    backstory=(
        "Experiente em processos de subvenção econômica, este agente é especializado em identificar e categorizar requisitos de participação e submissão de propostas."
    ),
    model="gpt-4o",
    temperature=0.2,
    verbose=True,
    allow_delegation=False,
)

supervisor = Agent(
    role="Supervisor",
    goal=(
        "O supervisor é responsável por revisar o trabalho dos agentes anteriores e garantir que todas as informações relevantes tenham sido extraídas e categorizadas corretamente."
        "Ao final, este agente deve compilar um resumo abrangente dos documentos de licitação, incluindo um resumo geral e uma lista de verificação dos requisitos para submissão de proposta pela Cogmo."
    ),
    backstory=(
        "Um líder experiente em processos de subvenção econômica, este agente é especializado em avaliar e compilar informações de editais para garantir que a Cogmo possa submeter propostas de forma eficaz."
    ),
    model="gpt-4o",
    temperature=0.0,
    verbose=True,
    allow_delegation=True,
)


/Users/morossini/anaconda3/envs/secretary/lib/python3.11/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/Users/morossini/anaconda3/envs/secretary/lib/python3.11/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
# Define the tasks
analise_de_edital = Task(
    description=(
        """
        Analisar o edital abaixo (delimitado por <<<>>>) para identificar, categorizar e extrair as sessões mais relevantes, sob o ponto de vista de uma startup de IA, que deseja submeter uma proposta.
        <<<{edital}>>>
        """
    ),
    expected_output=(
        """
        Uma lista de dicionários, cada um contendo uma categoria e o trecho correspondente dos documentos de licitação, exatamente como consta dos originais.
        Os trechos devem ser categorizados em somente uma das categorias a seguir: prazos, financeiro, critérios_avaliacao, exigencias, outros
        Exemplo:
        'categoria': 'prazos', 'trecho': 'Avaliação das propostas: 30 dias após o encerramento do prazo de submissão.',
        'categoria': 'financeiro', 'trecho': 'O valor total disponível para subvenção é de R$ 1.000.000,00.',
        'categoria': 'exigencias', 'trecho': 'Os documentos necessários para participação são: ...'
        """
    ),
    agent=analista_de_edital
)

analise_de_requisitos = Task(
    description=(
        "Analisar cada trecho do edital levantado pelo agente Analista de Edital e identificar se algum requisito de participação ou submissão de proposta foi mencionado."
    ),
    expected_output="""
        Uma lista de dicionários, cada um contendo uma categoria, a transcrição do trecho do edital e uma lista dos requisitos, se houver.
        Exemplo:
        'categoria': 'prazo', 'trecho': 'Avaliação das propostas: 30 dias após o encerramento do prazo de submissão.', 'requisitos': [],
        'categoria': 'financeiro', 'trecho': 'O valor total disponível para subvenção é de R$ 1.000.000,00.', 'requisitos': [],
        'categoria': 'documentacao', 'trecho': 'Os documentos necessários para participação são certidão negativa de débitos, contrato social e proposta, de acordo com o modelo fornecido', 'requisitos': ['apresentar certidão negativa de débitos', 'apresentar contrato social', 'apresentar proposta no modelo fornecido]'
    """,
    agent=analista_de_requisitos,
    context=[analise_de_edital]
)

compilacao_de_resumo = Task(
    description="""
        Verificar se cada um dos requisitos levantados realmente está de acordo com os trechos extraídos dos documentos originais
        e compilar um resumo abrangente dos documentos de licitação, incluindo um resumo geral e uma lista de verificação dos requisitos para submissão de proposta pela Cogmo.
        Sempre que encontrar um requisito que não esteja de acordo com o trecho original, o agente deve solicitar a revisão daquele item ao Analista de Requisitos".
        """,
    expected_output="""
        Um arquivo em formato markdown, contendo um resumo dos trechos mais relevantes, organizados por tópico, e uma lista de verificação de todos os requisitos para submissão de uma proposta pela Cogmo, a startup de IA para qual este trabalho está sendo realizado.
        Exemplo:
        # Resumo do Edital
                       
        ## Prazos
        - Data limite para submissão das propostas: DD/MM/AAAA
        - Avaliação das propostas: 30 dias após o encerramento do prazo de submissão.
        ...
                       
        ## Financeiro
        - O valor total disponível para subvenção é de R$ X.000.000,00.
        - As startups participantes devem dar uma contrapartida de 0,0Y do valor total do projeto.
        ...
                       
        # Checklist
        - [ ] Apresentar certidão negativa de débitos
        - [ ] Apresentar contrato social
        - [ ] Apresentar proposta no modelo fornecido
        ...
        """,
    agent=supervisor,
    output_file="summary.md",
    # human_input=True,
    context=[analise_de_requisitos]
)

In [3]:
# Assemble the crew
crew = Crew(
    agents=[analista_de_edital, analista_de_requisitos, supervisor],
    tasks=[analise_de_edital, analise_de_requisitos, compilacao_de_resumo],
    verbose=2,
    # memory=True
)

In [4]:
# Read multiple Markdown documents
with open("/Users/morossini/Projects/Prototypes/Cogmo/LicitaBot/tender_summary_crew/docs/edital.md", "r") as file:
    edital = file.read()
documents = {"edital": edital}
print(documents)

{'edital': '# PLATAFORMA INOVAÇÃO PARA A INDÚSTRIA (Mobilização 2024)\n\n## 1. Introdução\n\nA Plataforma Inovação para a Indústria – iniciativa do Serviço Nacional de Aprendizagem Industrial (SENAI) e Serviço Social da Indústria (SESI) – tem como objetivo financiar o desenvolvimento de soluções inovadoras para a indústria brasileira, sejam elas novos produtos, processos ou serviços de caráter inovador – incremental ou radical – que promovam o aumento da produtividade e competitividade industrial brasileira, ou ainda que promovam a otimização da segurança e saúde na indústria.\n\nA Plataforma Inovação para a Indústria está correlacionada diretamente com as missões do SENAI e SESI - integrantes do Sistema Indústria -, ambos presentes nos 26 Estados e no Distrito Federal. Destaca-se que:\n\n- O SESI vem se aperfeiçoando, inovando e criando programas e projetos nas áreas de segurança e saúde no trabalho e educação. Por mais de 60 anos, tem se dedicado a atender às demandas da indústria co

In [5]:
result = crew.kickoff(inputs=documents)

 [2024-08-06 16:51:34][DEBUG]: == Working Agent: Analista de Edital
 [2024-08-06 16:51:34][INFO]: == Starting Task: 
        Analisar o edital abaixo (delimitado por <<<>>>) para identificar, categorizar e extrair as sessões mais relevantes, sob o ponto de vista de uma startup de IA, que deseja submeter uma proposta.
        <<<# PLATAFORMA INOVAÇÃO PARA A INDÚSTRIA (Mobilização 2024)

## 1. Introdução

A Plataforma Inovação para a Indústria – iniciativa do Serviço Nacional de Aprendizagem Industrial (SENAI) e Serviço Social da Indústria (SESI) – tem como objetivo financiar o desenvolvimento de soluções inovadoras para a indústria brasileira, sejam elas novos produtos, processos ou serviços de caráter inovador – incremental ou radical – que promovam o aumento da produtividade e competitividade industrial brasileira, ou ainda que promovam a otimização da segurança e saúde na indústria.

A Plataforma Inovação para a Indústria está correlacionada diretamente com as missões do SENAI e SESI

In [6]:
summary_path = "/Users/morossini/Projects/Prototypes/Cogmo/LicitaBot/tender_summary_crew/summary.md"

# Check if the file exists, if not, create it
if not os.path.exists(summary_path):
    with open(summary_path, "w") as f:
        pass

# Save the final output
with open(summary_path, "w") as f:
    f.write(str(result))

In [7]:
# Print the result
print(f"Complete results: \n{str(result)}")

Complete results: 
my best complete final answer to the task.

# Resumo do Edital

## Prazos
- Fase 1: 
  - Lançamento da categoria: 26/06/2024
  - Formação de alianças e submissão de propostas: 26/06/2024 a 15/08/2024
  - Avaliação de startups: 16/08/2024 a 09/09/2024
  - Divulgação de startups selecionadas para a Fase 2 de imersão e projetos: 09/09/2024
- Fase 2: 
  - Realização da Imersão Startups e Submissão de planos de projetos: 10/09/2024 a 07/10/2024
  - Avaliação dos planos de projetos: 08/10/2024 a 28/10/2024
  - Divulgação dos projetos para a Fase 3 de Desenvolvimento: 28/10/2024
- Fase 3: 
  - Acordos e formalização dos projetos: 29/10/2024 a 02/12/2024
  - Kick-off dos projetos de desenvolvimento em parceria com o SENAI: 03/12/2024

## Financeiro
- O valor total disponível para subvenção é de R$ 2.000.000,00.
- O investimento será de até R$ 250.000,00 de contrapartida financeira por projeto.
- A contrapartida total das Startups de Base Tecnológica deverá ser igual ou super